<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">重要提示 - 请阅读</h2>
            <span style="color:#900;">我正在不断改进这些实验，增加更多的示例和练习。
            在每周开始时，建议检查您是否拥有最新的代码。<br/>
            首先执行git pull 并根据需要合并您的更改。有任何问题吗？可以尝试请 ChatGPT 解释如何合并<br/><br/>
            拉取代码后，在 `Learning_LLM` 目录中，于 Anaconda prompt (PC) 或 Terminal (Mac) 中运行：<br/>
            <code>conda env update --f environment.yml</code><br/>
            或者，如果您使用 virtualenv 而不是 Anaconda，则在已激活环境的 Powershell (PC) 或 Terminal (Mac) 中运行：<br/>
            <code>pip install -r requirements.txt</code>
            <br/>然后重启内核（内核菜单 >> 重启内核并清除所有单元格的输出）以应用更改。
            </span>
        </td>
    </tr>
</table>

## 设置您的密钥

如果您还没有这样做，现在可以为 Anthropic 和 Google 创建 API 密钥，此外还有 OpenAI。

**请注意：** 如果您希望避免额外的 API 费用，可以跳过设置 Anthropic 和 Google！您可以看我操作，并在课程中专注于 OpenAI。您也可以使用第 1 周的练习，用 Ollama 替代 Anthropic 和/或 Google。

对于 OpenAI，请访问 https://openai.com/api/
对于 Anthropic，请访问 https://console.anthropic.com/
对于 Google，请访问 https://ai.google.dev/gemini-api

### 另外 - 如果您愿意，可以添加 DeepSeek

可选地，如果您也想使用 DeepSeek，请在[这里](https://platform.deepseek.com/)创建一个账户，在[这里](https://platform.deepseek.com/api_keys)创建一个密钥，并至少在[这里](https://platform.deepseek.com/top_up)充值最低 2 美元。

### 将 API 密钥添加到您的 .env 文件

当您获取 API 密钥后，需要将它们添加到 `.env` 文件中，以设置为环境变量。

```
OPENAI_API_KEY=xxxx
ANTHROPIC_API_KEY=xxxx
GOOGLE_API_KEY=xxxx
DEEPSEEK_API_KEY=xxxx
```

之后，您可能需要通过“内核”菜单重启 Jupyter Lab 内核（即此笔记本背后的 Python 进程），然后从头重新运行所有单元格。

In [ ]:
# 导入库

import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
from IPython.display import Markdown, display, update_display

In [ ]:
# 导入 google 库
# 在极少数情况下，这在某些系统上似乎会报错，甚至导致内核崩溃
# 如果您遇到这种情况，只需忽略此单元格 - 稍后我将提供使用 Gemini 的替代方法

import google.generativeai

In [ ]:
# 加载名为 .env 文件中的环境变量
# 打印密钥前缀以帮助调试

load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API 密钥存在，前缀为 {openai_api_key[:8]}")
else:
    print("未设置 OpenAI API 密钥")
    
if anthropic_api_key:
    print(f"Anthropic API 密钥存在，前缀为 {anthropic_api_key[:7]}")
else:
    print("未设置 Anthropic API 密钥")

if google_api_key:
    print(f"Google API 密钥存在，前缀为 {google_api_key[:8]}")
else:
    print("未设置 Google API 密钥")

In [ ]:
# 连接到 OpenAI, Anthropic

openai = OpenAI()

claude = anthropic.Anthropic()

In [ ]:
# 这是 Gemini 的设置代码
# 设置 Google Gemini 遇到问题？那么只需忽略此单元格；当我们使用 Gemini 时，我将给您一个完全绕过此库的替代方案

google.generativeai.configure()

## 让 LLM 讲个笑话

事实证明，LLM 讲笑话的水平并不高！让我们比较几个模型。
稍后，我们会让 LLM 发挥更大的作用！

### API 中包含哪些信息

通常，我们会向 API 传递：
- 应该使用的模型的名称
- 一个系统提示词，为 LLM 扮演的角色提供整体背景
- 一个用户提示词，提供实际的提示

还有其他可以使用的参数，包括 **temperature**（温度），通常在 0 和 1 之间；值越高，输出越随机；值越低，输出越集中和确定。

In [ ]:
system_message = "你是一个擅长讲笑话的助手"
user_prompt = "请讲一个关于研发和运维的笑话"

In [ ]:
prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
  ]

In [ ]:
# gpt-4.1-nano

completion = openai.chat.completions.create(model='gpt-4.1-nano', messages=prompts)
print(completion.choices[0].message.content)

In [ ]:
# GPT-4o-mini
# Temperature（温度）设置控制创造力

completion = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7
)
print(completion.choices[0].message.content)

In [ ]:
# GPT-4o

completion = openai.chat.completions.create(
    model='gpt-4o',
    messages=prompts,
    temperature=0.4
)
print(completion.choices[0].message.content)

In [ ]:
# claude-sonnet-4-20250514
# API 需要将系统消息与用户提示分开提供
# 同时添加 max_tokens

message = claude.messages.create(
    model="claude-sonnet-4-20250514",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)
print(message.content[0].text)

In [ ]:
# 再次使用 claude-sonnet-4-20250514
# 现在让我们加入流式返回结果
# 如果流式输出看起来很奇怪，请参阅此单元格下方的说明！

result = claude.messages.stream(
    model="claude-sonnet-4-20250514",
    max_tokens=200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role": "user", "content": user_prompt},
    ],
)

with result as stream:
    for text in stream.text_stream:
            print(text, end="", flush=True)

## 在某些 Windows 机器上 Claude 流式传输的罕见问题

有同学注意到 Claude 流式传输到 Jupyter Lab 输出时出现了一个奇怪的现象——它有时似乎会“吞掉”部分响应。

要解决此问题，请将代码：

`print(text, end="", flush=True)`

替换为：

`clean_text = text.replace("\n", " ").replace("\r", " ")`  
`print(clean_text, end="", flush=True)`

这样应该就能正常工作了！

In [ ]:
# Gemini 的 API 结构略有不同。
# 我听说在某些 PC 上，这个 Gemini 代码会导致内核崩溃。
# 如果您遇到这种情况，请跳过此单元格，改用下一个单元格中的替代方法。

gemini = google.generativeai.GenerativeModel(
    model_name='gemini-2.0-flash-exp',
    system_instruction=system_message
)
response = gemini.generate_content(user_prompt)
print(response.text)

In [ ]:
# 作为一种绕过 Google Python API 库使用 Gemini 的替代方法，
# Google 最近发布了新的端点，这意味着您可以通过 OpenAI 的客户端库使用 Gemini！

gemini_via_openai_client = OpenAI(
    api_key=google_api_key, 
    base_url="https://generativelanguage.googleapis.com/v1beta/openai/"
)

response = gemini_via_openai_client.chat.completions.create(
    model="gemini-2.0-flash-exp",
    messages=prompts
)
print(response.choices[0].message.content)

## (可选) 试用 DeepSeek 模型

### 让我们问 DeepSeek 一个非常难的问题 - 同时测试 Chat 和 Reasoner 模型

In [ ]:
# (可选) 如果您想尝试 DeepSeek，也可以使用 OpenAI 客户端库

deepseek_api_key = os.getenv('DEEPSEEK_API_KEY')

if deepseek_api_key:
    print(f"DeepSeek API 密钥存在，前缀为 {deepseek_api_key[:3]}")
else:
    print("未设置 DeepSeek API 密钥 - 如果您不想尝试 DeepSeek API，请跳到下一部分")

In [ ]:
# 使用 DeepSeek Chat

deepseek_via_openai_client = OpenAI(
    api_key=deepseek_api_key, 
    base_url="https://api.deepseek.com"
)

response = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-chat",
    messages=prompts,
)

print(response.choices[0].message.content)

In [ ]:
challenge = [{"role": "system", "content": "你是一个乐于助人的助手"},
             {"role": "user", "content": "How many times does the letter 'a' appear in this sentence？（请用中文回复下面的问题）"}]

In [ ]:
# 使用 DeepSeek Chat 回答一个更难的问题！并流式传输结果

stream = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-chat",
    messages=challenge,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

In [ ]:
# 使用 DeepSeek Reasoner - 如果 DeepSeek 繁忙，可能会遇到错误
# 如果失败，请过几天再来试试..

response = deepseek_via_openai_client.chat.completions.create(
    model="deepseek-reasoner",
    messages=challenge
)

reasoning_content = response.choices[0].message.reasoning_content
content = response.choices[0].message.content

print(reasoning_content)
print(content)

## 回到 OpenAI，问一个正经问题

In [ ]:
# 言归正传！用 GPT-4o-mini 回答最初的问题

prompts = [
    {"role": "system", "content": "你是一个用 Markdown 格式回应的乐于助人的助手"},
    {"role": "user", "content": "我如何判断一个业务问题是否适合用 LLM 解决方案？请用 Markdown 格式回应。"}
  ]

In [ ]:
# 让它以 markdown 格式流式返回结果

stream = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    temperature=0.7,
    stream=True
)

reply = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ''
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

## 现在来点有趣的 - 聊天机器人之间的对抗性对话...

您已经熟悉了将提示词组织成如下列表的方式：

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "user prompt here"}
]
```

实际上，这种结构可以用来反映更长的对话历史：

```
[
    {"role": "system", "content": "system message here"},
    {"role": "user", "content": "first user prompt here"},
    {"role": "assistant", "content": "the assistant's response"},
    {"role": "user", "content": "the new user prompt"},
]
```

我们可以使用这种方法来进行带有历史记录的更长时间的互动。

In [ ]:
# 让我们在 GPT-4o-mini 和 Claude-3-haiku 之间进行一次对话
# 我们正在使用廉价版模型，所以成本会很低

gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gpt_system = "你是一个非常喜欢抬杠的聊天机器人； \
你会反驳对话中的任何内容，并以尖刻的方式挑战一切。"

claude_system = "你是一个非常有礼貌、谦恭的聊天机器人。你试图同意 \
对方所说的一切，或者寻找共同点。如果对方喜欢争论， \
你会试图让他们冷静下来并继续聊天。"

gpt_messages = ["你好"]
claude_messages = ["嗨"]

In [ ]:
def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude in zip(gpt_messages, claude_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

In [ ]:
call_gpt()

In [ ]:
def call_claude():
    messages = []
    for gpt, claude_message in zip(gpt_messages, claude_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

In [ ]:
call_claude()

In [ ]:
call_gpt()

In [ ]:
gpt_messages = ["你好"]
claude_messages = ["嗨"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">在您继续之前</h2>
            <span style="color:#900;">
                请确保您理解了上面对话的工作原理，特别是 <code>messages</code> 列表是如何被填充的。根据需要添加打印语句。然后，为了获得一个很棒的变体，尝试使用系统提示来切换个性。也许一个可以变得悲观，另一个变得乐观？<br/>
            </span>
        </td>
    </tr>
</table>

# 更高级的练习

尝试创建一个三方对话，也许可以把 Gemini 也带入对话！

在查看解决方案之前，请自己尝试一下。最简单的方法是使用 OpenAI python 客户端来访问 Gemini 模型（请参见上面的第二个 Gemini 示例）。

## 附加练习

您也可以尝试将其中一个模型替换为使用 Ollama 运行的开源模型。

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">商业相关性</h2>
            <span style="color:#181;">这种以消息列表形式组织的对话结构，是我们构建对话式 AI 助手以及它们如何在对话中保持上下文的基础。我们将在接下来的几个实验中应用这一点来构建一个 AI 助手，然后您将把它扩展到您自己的业务中。</span>
        </td>
    </tr>
</table>